# Error handling

ironArray has a sophisticated error handling machinery allowing to catch erros at multiple levels. To see how all this works, let's create a very simple array and deal with some invalid copies:

In [1]:
import iarray as ia
import numpy as np

In [2]:
dtshape = ia.DTShape((1000, 1000), dtype=np.float32)
a = ia.linspace(dtshape, 0, 1)

In [3]:
with ia.config(chunkshape=(100, 100), blockshape=(50, 50)) as cfg:
    a.copy(cfg=cfg)


As we can see, this first copy was created correctly. But what happens if we create a copy with a chunkshape larger than the shape of the array?

In [4]:
with ia.config(chunkshape=(10000, 10000), blockshape=(1000, 1000)) as cfg:
    a.copy(cfg=cfg)


IArrayError: b'CHUNK SHAPE NOT VALID - 0x800a000000938404 - error=1,ver=0,rev=10,os=0,neg=1,adj=147,subject=1028,code=9633792,ubits=0x0'

The ironArray error handling machinery raised a Python error called ``IArrayError``. This error returns a general error message. In this particular case, the error is ``CHUNK SHAPE NOT VALID`` but it does not indicate much more beyond this.

In order to obtain more detailed information, we can set the environment variable ``INAC_TRACE="iarray.error"``. With this, ironArray will show us a more detailed error; in particular, it will provide us with a trace of what's going on in the underlying C library.

To better leverage this functionality inside a Jupyter notebook, it is convenient to use the ``wurlitzer`` extension to redirect `stdout` and `stderr` handlers to the notebook:

In [5]:
%load_ext wurlitzer

import os
os.environ['INAC_TRACE'] = 'iarray.error'

and, after doing this, we can get more info on the ironArray core library (written in C):

In [6]:
with ia.config(chunkshape=(10000, 10000), blockshape=(1000, 1000)) as cfg:
    a.copy(cfg=cfg)

[iarray.error] - The chunkshape is bigger than shape /Users/faltet/inaos/iron-array-python/iarray/iarray-c-develop/src/iarray_container.c:271
[iarray.error] - Tracing:  /Users/faltet/inaos/iron-array-python/iarray/iarray-c-develop/src/iarray_constructor.c:458


IArrayError: b'CHUNK SHAPE NOT VALID - 0x800a000000938404 - error=1,ver=0,rev=10,os=0,neg=1,adj=147,subject=1028,code=9633792,ubits=0x0'

Now, we get the error message ``The chunkshape is bigger than shape`` that already gives us a better idea of what we were doing wrong.

In addition, we can see in which lines of the ironArray core code the error has
occurred. In this case, the trace is:

- ``iarray/iarray-c-develop/src/iarray_container.c:271``
- ``iarray/iarray-c-develop/src/iarray_constructor.c:458``

While those will be mostly useful for developers, it is always interesting that you can attach these  when you file possible bug reports.

That's all.  Whenever you would like more info on your errors, remember to activate the `INAC_TRACE` environment variable.